__This notebook has been made to reverse engineer the datasets, since no specific methods or techniques are explained within the paper.__

> The original datasets have been downloaded together with the datasets provided by the authors.

> The recreated datasets have been standardized and some assumptions have been made to for example make attributes binary, one-hot encode categorical features and make the targets binary as well.

> Our recreated data has been compared with the datasets provided by the authors, to check whether these transformations/techniques can be applied to reproduce their paper/results.


# Table Of Contents:
1. [Helper functions to create datasets](#helpers)
2. [German credit dataset](#german)
2.1. [Original/recreated dataset](#german-orig)
2.2. [Authors dataset](#german-author)
2.3. [Reverse engineering](#german-reverse)
3. [Drug consumption dataset](#drug)
3.1. [Original/recreated dataset](#drug-orig)
3.2. [Authors dataset](#drug-author)
3.3. [Reverse engineering](#drug-reverse)
4. [Compas dataset](#compas)
4.1. [Original/recreated dataset](#compas-orig)
4.2. [Authors dataset](#compas-author)
4.3. [Reverse engineering](#compas-reverse)



In [27]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# 1. Helper functions to create datasets  <a class="anchor" id="helpers"></a>

In [28]:
def make_df_orig(file_path, sensitive_feature_idx, used_columns=None, categorical_variables=False,
                 stop_index=None, target_included=False, target_idx=None):
    
    ### datafames use different delimiters, so need to check for name in pathfile ###
    
    # in the original drug data they only use the first 13 columns
    if "drug" in file_path:
        df = pd.read_csv(file_path, delimiter=",", header=None)
        df = df.iloc[:, :stop_index]
        
    
    
    # sensitive feature (making binary)
    if "german" in file_path:
        df = pd.read_csv(file_path, delim_whitespace=True, header=None)
        
        # if target is included in original df, drop from df for now
        if target_included:
            df = df.drop(target_idx, axis=1)
            
        df[sensitive_feature_idx] = df[sensitive_feature_idx].replace({"A91": 0, "A92": 1, 
                                                                       "A93": 0, "A94": 0, "A95":1})
        
        
    # uses a specific selection of column names and sensitive feature must be binary
    if "compas" in file_path:
        df = pd.read_csv(file_path)
        df = df[used_columns]
        df[sensitive_feature_idx] = df[sensitive_feature_idx].replace({"Male":0, "Female":1})
        
    if categorical_variables:
        # save indices of categecorical variables
        str_columns = [i for i in df.columns if df[i].dtype == object]
        not_str = [i for i in df.columns if df[i].dtype != object]

        # one-hot encode cat. variables and concatenate with rest of dataframe
        dummies = pd.get_dummies(df[str_columns])
        df = pd.concat([df[not_str], dummies], axis=1, join='inner')
    
    
    # make new dataframe with only integers as column names and rescale to mean = 0 and std = 1
    # sort df to be able to compare (we do not do that for german)
    if not "german" in file_path:
        df = df.rename(columns={i:j for i,j in zip(df.columns, range(len(df.columns)))})
        
    for i in df.columns:
        df[i] = np.sort(preprocessing.scale(df[i]))
    
    return df

In [29]:
def make_df_authors(file_path):
    data = np.load(file_path)
    data = np.concatenate((data["X_train"], data["X_test"]))
    
    df = pd.DataFrame(data=data)
    
    # sort values in df to be able to compare
    for i in df.columns:
        df[i] = np.sort(df[i])
    
    return df

# 2. German credit dataset <a class="anchor" id="german"></a>

## 2.1. Original/recreated dataset <a class="anchor" id="german-orig"></a>

We will first show the original dataset without modifications. We got it from the [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) as mentioned in the paper.

In [30]:
file_path = "./resources/german.data"
data = pd.read_csv(file_path, delim_whitespace=True, header=None)
data = data.drop(20, axis=1) ## drop TARGET labels
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201


__We have to modify this dataset a little bit:__
1. As we can see, there are many categorical variables. These have to be one-hot encoded. 
2. We know from [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) that the sensitive feature is at the 8th index (sex). See explanation below.


    Attribute 9 (in our dataset attr  and index 8. Since our idx starts at 0):
    Personal status and sex 
    A91 : male : divorced/separated 
    A92 : female : divorced/separated/married 
    A93 : male : single 
    A94 : male : married/widowed 
    A95 : female : single 

**We will make it binary as following:** $\left\{\begin{array}{l}1 \text { if } x \in \{A92, A95\} \\ 0 \text { otherwise }\end{array}\right.$

In [31]:
file_path = "./resources/german.data"
german_original = make_df_orig(file_path, 8, categorical_variables=True, target_included=True, target_idx=20)
german_original

,1,4,7,8,10,12,15,17,0_A11,0_A12,...,14_A152,14_A153,16_A171,16_A172,16_A173,16_A174,18_A191,18_A192,19_A201,19_A202
0,-1.402415,-1.070865,-1.764514,-0.670280,-1.672459,-1.455261,-0.704926,-0.428290,-0.614337,-0.606621,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
1,-1.402415,-1.061649,-1.764514,-0.670280,-1.672459,-1.455261,-0.704926,-0.428290,-0.614337,-0.606621,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
2,-1.402415,-1.039674,-1.764514,-0.670280,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
3,-1.402415,-1.039319,-1.764514,-0.670280,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
4,-1.402415,-1.037902,-1.764514,-0.670280,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.243815,4.388626,0.918477,1.491914,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
996,3.243815,4.395361,0.918477,1.491914,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
997,3.243815,4.460933,0.918477,1.491914,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
998,3.243815,4.492124,0.918477,1.491914,1.046987,3.470076,4.491089,2.334869,1.627770,1.648476,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669


## 2.2. Authors dataset <a class="anchor" id="german-author"></a>

In [32]:
file_path = './authors_data/data.npz'
german_authors = make_df_authors(file_path)
german_authors

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-1.402415,-1.070865,-1.764514,-1.672459,-1.455261,-0.704926,-0.428290,-0.614337,-0.606621,-0.259299,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
1,-1.402415,-1.061649,-1.764514,-1.672459,-1.455261,-0.704926,-0.428290,-0.614337,-0.606621,-0.259299,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
2,-1.402415,-1.039674,-1.764514,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,-0.259299,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
3,-1.402415,-1.039319,-1.764514,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,-0.259299,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
4,-1.402415,-1.037902,-1.764514,-1.672459,-1.367309,-0.704926,-0.428290,-0.614337,-0.606621,-0.259299,...,-1.576173,-0.347960,-0.149983,-0.5,-1.304877,-0.416784,-1.214598,-0.823318,-5.101669,-0.196014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.243815,4.388626,0.918477,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,3.856555,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
996,3.243815,4.395361,0.918477,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,3.856555,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
997,3.243815,4.460933,0.918477,1.046987,3.382124,4.491089,2.334869,1.627770,1.648476,3.856555,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669
998,3.243815,4.492124,0.918477,1.046987,3.470076,4.491089,2.334869,1.627770,1.648476,3.856555,...,0.634448,2.873893,6.667424,2.0,0.766356,2.399324,0.823318,1.214598,0.196014,5.101669


As we can see already, the datasets are **matching almost**. Only **<font color='red'>a small difference in the order</font>** of the **<font color='red'>columns</font>**.

## 2.3. Reverse engineering <a class="anchor" id="german-reverse"></a>

To be able to reverse engineer it, we have already done a couple of transformation (which can be seen in the [first helper](#helpers) function). Some of them are:
1. Made the sex attribute binary which was categorical, using females as 1 and males as 0.
2. One-hot encoded the categorical attributes.
3. Standardized the dataset (mean = 0 and std = 1, unit variance).

Now we will show that our reverse engineering is correct.

First we will check **which idx in the authors data is the sensitive feature idx**. We do this by checking which column is equal to our sensitive feature column (since we know which column is the sensitive feature). 

In [33]:
german_original[8] # Gender
for i in german_authors.columns:
    if np.allclose(german_original[8], german_authors[i], atol=1e-08):
        print("Column:", german_original[8].name, "matches column:", german_authors.columns[i])

Column: 8 matches column: 36


__Thus our reverse engineering is **<font color='green'>correct</font>**. They used feature 36 as the sensitive feature IDX.__ But to be sure that all other columns are equal, we can loop through both dataframes, check which 2 columns are equal and pop them both until we do not have any columns left.


In [34]:
######## just here because of the pop, otherwise we have to run the code from the top again ########
file_path = "./resources/german.data"
german_original = make_df_orig(file_path, 8, categorical_variables=True, target_included=True, target_idx=20)

# now we are renaming all the columns to integers from 0 up to length of the df
german_original = german_original.rename(columns={i:j for i,j in zip(german_original.columns, 
                                                                     range(len(german_original.columns)))})

file_path = './authors_data/data.npz'
german_authors = make_df_authors(file_path)
######################################################################################################   

counter1, counter2 = 0, 0
p = len(german_original.columns)

while p:
    # check if two columns are matching
    if np.allclose(german_original[counter1], german_authors[counter2]):
        # pop from both dataframes
        german_original.pop(counter1), german_authors.pop(counter2)
        
        # rename both dataframes, because we are working with counters (otherwise we would get index errors)
        german_original = german_original.rename(columns={i:j for i,j in zip(german_original.columns, 
                                                                             range(len(german_original.columns)))})
        german_authors = german_authors.rename(columns={i:j for i,j in zip(german_authors.columns, 
                                                                           range(len(german_authors.columns)))})
        
        # set both counters to 0 (otherwise we would get out of index errors)
        counter1, counter2 = 0, 0
        
        # set the while condition again
        p = len(german_original.columns)
    
    else:
        # if no match, increase the second counter (so per column of df1 loop through the entire df2)
        counter2 += 1
        
print("The two dataframes are equal, since the matching columns are popped one by one from both dataframes.   \
       Otherwise, the while loop would never stop.")

The two dataframes are equal, since the matching columns are popped one by one from both dataframes.          Otherwise, the while loop would never stop.


**To check how the group labels and targets are made, we can do the following:**

In [35]:
data = np.load('./authors_data/data.npz')
group_labels = np.load('./authors_data/german_group_label.npz')

y_data = np.concatenate((data["Y_train"], data["Y_test"]))
y_df = pd.DataFrame(data=y_data, columns={58}).rename(columns={58:"targets"})

x_data = np.concatenate((data["X_train"], data["X_test"]))
x_df = pd.DataFrame(data=x_data)

group_labels_df = pd.DataFrame(data = group_labels["group_label"], columns={"group_labels"})

full_df = pd.concat([x_df, y_df, group_labels_df], axis=1, join="inner")
selection_df = full_df[[36, "targets", "group_labels"]]  ## 36 is the sex attribute in their dataset
selection_df.head(10)

,36,targets,group_labels
0,-0.670280,0,0
1,1.491914,1,1
2,-0.670280,0,0
3,-0.670280,0,0
4,-0.670280,1,0
5,-0.670280,0,0
6,-0.670280,0,0
7,-0.670280,0,0
8,-0.670280,0,0
9,-0.670280,1,0


**So we can see that the:**
- **Group labels**: are the advantaged/disadvantaged groups, i.e. male/female
- **Targets**: are the targets, i.e. whether an individual has good or bad credit.

**Furthermore we can still make these assumptions (<font color='red'>red</font>) and claim that we have tested some of our assumptions (<font color='green'>green</font>):**
- <font color='red'> They shuffled the dataset first </font>
- <font color='green'> One-hot encoded the categorical variables </font>
- <font color='green'> Made the sex attribute binary which was categorical, using females as 1 and males as 0 </font>
- <font color='green'> Standardized dataset </font>
- <font color='green'> Played around with the order of the columns/features </font>
- <font color='green'> Used feature 36 as feature idx </font>

# 3. Drug consumption dataset  <a class="anchor" id="drug"></a>

## 3.1. Original/recreated dataset <a class="anchor" id="drug-orig"></a>

We will first show the original dataset without modifications. We got it from the [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29) as mentioned in the paper.

In [36]:
file_path = "./resources/drug_consumption.data"
data = pd.read_csv(file_path, delimiter=",", header=None)
data = data.iloc[:, :13]
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575


__This dataset we do not have to modify.__
We know from the [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29) that column 2 (idx 2 as well) is the sensitive feature (gender).

In [37]:
file_path = "./resources/drug_consumption.data"
drug_original = make_df_orig(file_path, 2, stop_index=13, target_included=False)
drug_original

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-1.732578,-1.123504,-0.999470,-2.560578,-1.322050,-4.798619,-3.471902,-3.283013,-3.286660,-3.473926,-3.473498,-2.685502,-2.153924
1,-1.730743,-1.123504,-0.999470,-2.560578,-1.322050,-4.798619,-3.164228,-3.283013,-3.286660,-3.166046,-3.165644,-2.685502,-2.153924
2,-1.728909,-1.123504,-0.999470,-2.560578,-1.322050,-4.798619,-2.762971,-3.013694,-2.870550,-3.013636,-2.909201,-2.685502,-2.153924
3,-1.727074,-1.123504,-0.999470,-2.560578,-1.322050,-4.798619,-2.762971,-2.735812,-2.870550,-2.909582,-2.909201,-2.685502,-2.153924
4,-1.725239,-1.123504,-0.999470,-2.560578,-1.322050,-4.798619,-2.762971,-2.735812,-2.870550,-2.795580,-2.909201,-2.685502,-2.153924
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1.722322,2.911992,1.000531,2.093200,0.864498,2.621090,2.828019,2.867739,2.913904,2.765014,2.639614,3.033379,1.998060
1881,1.724157,2.911992,1.000531,2.093200,0.864498,2.621090,2.828019,2.867739,2.913904,2.765014,3.014021,3.033379,1.998060
1882,1.725991,2.911992,1.000531,2.093200,0.864498,13.339778,2.828019,3.014021,2.913904,2.765014,3.014021,3.033379,1.998060
1883,1.727826,2.911992,1.000531,2.093200,0.864498,13.339778,3.280967,3.283339,2.913904,3.166538,3.014021,3.033379,1.998060


## 3.2. Authors dataset <a class="anchor" id="drug-author"></a>

In [38]:
file_path = './authors_data/drug2_data.npz'
drug_authors = make_df_authors(file_path)
drug_authors

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-1.732578,-1.123504,-2.560578,-1.322050,-4.798619,-3.471902,-3.283013,-3.286660,-3.473926,-3.473498,-2.685502,-2.153924,-0.999470
1,-1.730743,-1.123504,-2.560578,-1.322050,-4.798619,-3.164228,-3.283013,-3.286660,-3.166046,-3.165644,-2.685502,-2.153924,-0.999470
2,-1.728909,-1.123504,-2.560578,-1.322050,-4.798619,-2.762971,-3.013694,-2.870550,-3.013636,-2.909201,-2.685502,-2.153924,-0.999470
3,-1.727074,-1.123504,-2.560578,-1.322050,-4.798619,-2.762971,-2.735812,-2.870550,-2.909582,-2.909201,-2.685502,-2.153924,-0.999470
4,-1.725239,-1.123504,-2.560578,-1.322050,-4.798619,-2.762971,-2.735812,-2.870550,-2.795580,-2.909201,-2.685502,-2.153924,-0.999470
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1.722322,2.911992,2.093200,0.864498,2.621090,2.828019,2.867739,2.913904,2.765014,2.639614,3.033379,1.998060,1.000531
1881,1.724157,2.911992,2.093200,0.864498,2.621090,2.828019,2.867739,2.913904,2.765014,3.014021,3.033379,1.998060,1.000531
1882,1.725991,2.911992,2.093200,0.864498,13.339778,2.828019,3.014021,2.913904,2.765014,3.014021,3.033379,1.998060,1.000531
1883,1.727826,2.911992,2.093200,0.864498,13.339778,3.280967,3.283339,2.913904,3.166538,3.014021,3.033379,1.998060,1.000531


We can see immediately that the datasets are **matching except the sensitive feature (order in df)**. 

## 3.3. Reverse engineering <a class="anchor" id="drug-reverse"></a>

To be able to reverse engineer it, we have already done a couple of transformation (which can be seen in the [first helper](#helpers) function). Some of them are:
1. Standardized the dataset (mean = 0 and std = 1, unit variance).

First we will check **which idx in the authors data is the sensitive feature idx**. We do this by checking which column is equal to our sensitive feature column (since we know which column is the sensitive feature). 

In [39]:
drug_original[2] # Gender
for i in drug_authors.columns:
    if np.allclose(drug_original[2], drug_authors[i], atol=1e-08):
        print("Column:", drug_original.columns[2], "matches column:", drug_authors.columns[i])

Column: 2 matches column: 12


__This sensitive feature idx was not given. We found it by reverse engineering it.__ To be sure that all other columns are equal, we can loop through both dataframes, check which 2 columns are equal and pop them both until we do not have any columns left.

In [40]:
######## just here because of the pop, otherwise we have to run the code from the top again ########
file_path = "./resources/drug_consumption.data"
drug_original = make_df_orig(file_path, 2, stop_index=13, target_included=False)

file_path = './authors_data/drug2_data.npz'
drug_authors = make_df_authors(file_path)
###################################################################################################### 

counter1, counter2 = 0, 0
p = len(drug_original.columns)

while p:
    # check if two columns are matching
    if np.allclose(drug_original[counter1], drug_authors[counter2]):
        # pop from both dataframes
        drug_original.pop(counter1), drug_authors.pop(counter2)
        
        # rename both dataframes, because we are working with counters (otherwise we would get index errors)
        drug_original = drug_original.rename(columns={i:j for i,j in zip(drug_original.columns, 
                                                                         range(len(drug_original.columns)))})
        drug_authors = drug_authors.rename(columns={i:j for i,j in zip(drug_authors.columns, 
                                                                       range(len(drug_authors.columns)))})
        
        # set both counters to 0 (otherwise we would get out of index errors)
        counter1, counter2 = 0, 0
        
        # set the while condition again
        p = len(drug_original.columns)
    
    else:
        # if no match, increase the second counter (so per column of df1 loop through the entire df2)
        counter2 += 1
        
print("The two dataframes are equal, since the matching columns are popped one by one from both dataframes.   \
       Otherwise, the while loop would never stop.")

The two dataframes are equal, since the matching columns are popped one by one from both dataframes.          Otherwise, the while loop would never stop.


**To check how the group labels and targets are made, we can do the following:**

In [41]:
data = np.load('./authors_data/drug2_data.npz')
group_labels = np.load('./authors_data/drug2_group_label.npz')

y_data = np.concatenate((data["Y_train"], data["Y_test"]))
y_df = pd.DataFrame(data=y_data, columns={"targets"})

x_data = np.concatenate((data["X_train"], data["X_test"]))
x_df = pd.DataFrame(data=x_data)

group_labels_df = pd.DataFrame(data = group_labels["group_label"], columns={"group_labels"})

full_df = pd.concat([x_df, y_df, group_labels_df], axis=1, join="inner")
selection_df = full_df[[12, "targets", "group_labels"]]  ## 12 is the gender attribute in their dataset
selection_df.head(10)

,12,targets,group_labels
0,1.000531,1,1
1,-0.999470,0,0
2,1.000531,1,1
3,1.000531,1,1
4,1.000531,1,1
5,-0.999470,0,0
6,-0.999470,0,0
7,-0.999470,0,0
8,-0.999470,0,0
9,-0.999470,1,0


**So we can see that the:**
- **Group labels**: are the advantaged/disadvantaged groups, i.e. male/female
- **Targets**: are the targets, i.e. whether an individual has consumed cocaine or not in their lifetime.

**Furthermore we can still make these assumptions (<font color='red'>red</font>) and claim that we have tested some of our assumptions (<font color='green'>green</font>):**
- <font color='red'> They shuffled the dataset first </font>
- <font color='green'> Standardized dataset </font>
- <font color='green'> Played around with the order of the columns/features </font>
- <font color='green'> Used feature 12 as feature idx </font>

# 4. Compas dataset  <a class="anchor" id="compas"></a>

## 4.1. Original/recreated dataset <a class="anchor" id="compas-orig"></a>

We will first show the original dataset without modifications. We got it from [Propublica](https://github.com/propublica/compas-analysis/) and use the same columns as mentioned in the paper.

In [42]:
data = pd.read_csv("./resources/compas-scores-two-years.csv")
used_cols = ["sex", "juv_fel_count", "priors_count", "race", "age_cat", "juv_misd_count", "c_charge_degree", "juv_other_count"]
data = data[used_cols]
data.head()

,sex,juv_fel_count,priors_count,race,age_cat,juv_misd_count,c_charge_degree,juv_other_count
0,Male,0,0,Other,Greater than 45,0,F,0
1,Male,0,0,African-American,25 - 45,0,F,0
2,Male,0,4,African-American,Less than 25,0,F,1
3,Male,0,1,African-American,Less than 25,1,F,0
4,Male,0,2,Other,25 - 45,0,F,0


__Again we have to modify this dataset a little bit:__
1. As we can see, there are many categorical variables. These have to be one-hot encoded. 
2. We know from [UCI machine learning repository](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) that the sensitive feature is at the 0th index (sex). See explanation below.

**We will make the <font color='red'> feature idx </font> binary as following:** $\left\{\begin{array}{l}1 \text { if } female \\ 0 \text { otherwise }\end{array}\right.$

In [43]:
file_path = "./resources/compas-scores-two-years.csv"
used_cols = ["sex", "juv_fel_count", "priors_count", "race", "age_cat", 
             "juv_misd_count", "c_charge_degree", "juv_other_count"]
compas_original = make_df_orig(file_path, "sex", used_cols, categorical_variables=True, target_included=False)
compas_original

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.489624,-0.141855,-0.711240,-0.187414,-0.218065,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.150369,-0.528708,-0.518607,-1.353233,-0.738971
1,-0.489624,-0.141855,-0.711240,-0.187414,-0.218065,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.150369,-0.528708,-0.518607,-1.353233,-0.738971
2,-0.489624,-0.141855,-0.711240,-0.187414,-0.218065,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.150369,-0.528708,-0.518607,-1.353233,-0.738971
3,-0.489624,-0.141855,-0.711240,-0.187414,-0.218065,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.150369,-0.528708,-0.518607,-1.353233,-0.738971
4,-0.489624,-0.141855,-0.711240,-0.187414,-0.218065,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.150369,-0.528708,-0.518607,-1.353233,-0.738971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,2.042382,16.737967,6.457660,16.300466,11.744825,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.869286,1.891405,1.928242,0.738971,1.353233
7210,2.042382,16.737967,6.662485,16.300466,13.738640,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.869286,1.891405,1.928242,0.738971,1.353233
7211,2.042382,18.847944,6.867311,16.300466,13.738640,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.869286,1.891405,1.928242,0.738971,1.353233
7212,2.042382,20.957922,7.072137,24.544406,17.726270,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.869286,1.891405,1.928242,0.738971,1.353233


## 4.2. Authors dataset <a class="anchor" id="compas-author"></a>

In [44]:
file_path = './authors_data/compas_data.npz'
compas_authors = make_df_authors(file_path)
compas_authors

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.141855,-0.187414,-0.218065,-0.711240,-0.489624,-1.150369,-0.528708,-0.518607,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.353233,-0.738971
1,-0.141855,-0.187414,-0.218065,-0.711240,-0.489624,-1.150369,-0.528708,-0.518607,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.353233,-0.738971
2,-0.141855,-0.187414,-0.218065,-0.711240,-0.489624,-1.150369,-0.528708,-0.518607,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.353233,-0.738971
3,-0.141855,-0.187414,-0.218065,-0.711240,-0.489624,-1.150369,-0.528708,-0.518607,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.353233,-0.738971
4,-0.141855,-0.187414,-0.218065,-0.711240,-0.489624,-1.150369,-0.528708,-0.518607,-1.024986,-0.066750,-0.718015,-0.311212,-0.050014,-0.234822,-1.353233,-0.738971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,16.737967,16.300466,11.744825,6.457660,2.042382,0.869286,1.891405,1.928242,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.738971,1.353233
7210,16.737967,16.300466,13.738640,6.662485,2.042382,0.869286,1.891405,1.928242,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.738971,1.353233
7211,18.847944,16.300466,13.738640,6.867311,2.042382,0.869286,1.891405,1.928242,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.738971,1.353233
7212,20.957922,24.544406,17.726270,7.072137,2.042382,0.869286,1.891405,1.928242,0.975623,14.981238,1.392728,3.213248,19.994444,4.258554,0.738971,1.353233


As we can see already, the datasets are **matching almost**. Only **<font color='red'>a small difference in the order</font>** of the **<font color='red'>columns</font>**.

## 4.3. Reverse engineering  <a class="anchor" id="compas-reverse"></a>

To be able to reverse engineer it, we have already done a couple of transformation (which can be seen in the [first helper](#helpers) function). Some of them are:
1. Made the sex attribute binary which was categorical, using females as 1 and males as 0.
2. One-hot encoded the categorical attributes.
3. Standardized the dataset (mean = 0 and std = 1, unit variance).

First we will check **which idx in the authors data is the sensitive feature idx**. We do this by checking which column is equal to our sensitive feature column (since we know which column is the sensitive feature). 

In [45]:
compas_original[0] # sex
for i in compas_authors.columns:
    if np.allclose(compas_original[0], compas_authors[i], atol=1e-08):
        print("Column:", compas_original.columns[0], "matches column:", compas_authors.columns[i])

Column: 0 matches column: 4


__This sensitive feature idx was not given. We found it by reverse engineering it.__ To be sure that all other columns are equal, we can loop through both dataframes, check which 2 columns are equal and pop them both until we do not have any columns left.

In [46]:
######## just here because of the pop, otherwise we have to run the code from the top again ########
file_path = "./resources/compas-scores-two-years.csv"
used_cols = ["sex", "juv_fel_count", "priors_count", "race", "age_cat", 
             "juv_misd_count", "c_charge_degree", "juv_other_count"]
compas_original = make_df_orig(file_path, "sex", used_cols, categorical_variables=True, target_included=False)

file_path = './authors_data/compas_data.npz'
compas_authors = make_df_authors(file_path)
######################################################################################################

counter1, counter2 = 0, 0
p = len(compas_original.columns)

while p:
    # check if two columns are matching
    if np.allclose(compas_original[counter1], compas_authors[counter2]):
        # pop from both dataframes
        compas_original.pop(counter1), compas_authors.pop(counter2)
        
        # rename both dataframes, because we are working with counters (otherwise we would get index errors)
        compas_original = compas_original.rename(columns={i:j for i,j in zip(compas_original.columns, 
                                                                         range(len(compas_original.columns)))})
        compas_authors = compas_authors.rename(columns={i:j for i,j in zip(compas_authors.columns, 
                                                                       range(len(compas_authors.columns)))})
        
        # set both counters to 0 (otherwise we would get out of index errors)
        counter1, counter2 = 0, 0
        
        # set the while condition again
        p = len(compas_original.columns)
    
    else:
        # if no match, increase the second counter (so per column of df1 loop through the entire df2)
        counter2 += 1
        
print("The two dataframes are equal, since the matching columns are popped one by one from both dataframes.   \
       Otherwise, the while loop would never stop.")

The two dataframes are equal, since the matching columns are popped one by one from both dataframes.          Otherwise, the while loop would never stop.


**To check how the group labels and targets are made, we can do the following:**

In [48]:
data = np.load('./authors_data/compas_data.npz')
group_labels = np.load('./authors_data/compas_group_label.npz')

y_data = np.concatenate((data["Y_train"], data["Y_test"]))
y_df = pd.DataFrame(data=y_data, columns={"targets"})

x_data = np.concatenate((data["X_train"], data["X_test"]))
x_df = pd.DataFrame(data=x_data)

group_labels_df = pd.DataFrame(data = group_labels["group_label"], columns={"group_labels"})

full_df = pd.concat([x_df, y_df, group_labels_df], axis=1, join="inner")
selection_df = full_df[[4, "targets", "group_labels"]]  ## 4 is the gender attribute in their dataset
selection_df.head(10)

,4,targets,group_labels
0,-0.489624,0,0
1,-0.489624,1,0
2,-0.489624,1,0
3,-0.489624,0,0
4,-0.489624,0,0
5,-0.489624,0,0
6,-0.489624,1,0
7,-0.489624,0,0
8,2.042382,0,1
9,-0.489624,1,0


**So we can see that the:**
- **Group labels**: are the advantaged/disadvantaged groups, i.e. male/female
- **Targets**: are the targets, i.e. whether an individual will recommit a crime within two years.

**Furthermore we can still make these assumptions (<font color='red'>red</font>) and claim that we have tested some of our assumptions (<font color='green'>green</font>):**
- <font color='red'> They shuffled the dataset first </font>
- <font color='green'> One-hot encoded the categorical variables </font>
- <font color='green'> Made the sex attribute binary which was categorical, using females as 1 and males as 0 </font>
- <font color='green'> Standardized dataset </font>
- <font color='green'> Played around with the order of the columns/features </font>
- <font color='green'> Used feature 4 as feature idx </font>